### Part I 
#### Data Cleaning, Wrandling, Additional information 

Importing Libraries

In [1]:
import pandas as pd
import os
import requests
import time

Loading data - Car accident data for Monroe County, from 2003 to 2015.

In [2]:
data = pd.read_csv("monroe-county-crash-data2003-to-2015.csv", encoding="unicode_escape")
# ========================
# preparing data
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)
# creation of variable with lon and lat together
data['ll'] = data['Latitude'].astype(str) + ',' + data['Longitude'].astype(str)
data = data[data['ll'] != '0.0,0.0']

#Modifying data format
data['Month'] = data['Month'].apply(lambda x: str(x).zfill(2))
data['Day'] = data['Day'].apply(lambda x: str(x).zfill(2))

data['Hour'] = data['Hour'].fillna(0)
data['Hour'] = data['Hour'].astype(int).astype(str).str.zfill(4)


data['Date'] = data ['Year'].astype(str)+'-'+ data['Month'].astype(str)+ '-' + data['Day'].astype(str)
data = data[data['Date'] != '0000-00-00']

print(data.shape)
data.head()

(49005, 14)


,Master Record Number,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,ll,Date
0,902363382,2015,01,05,Weekday,0000,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356",2015-01-05
1,902364268,2015,01,06,Weekday,1500,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848",2015-01-06
2,902364412,2015,01,06,Weekend,2300,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006",2015-01-06
3,902364551,2015,01,07,Weekend,0900,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635",2015-01-07
4,902364615,2015,01,07,Weekend,1100,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482",2015-01-07


In [3]:
data = data.rename(columns={'ll': 'Location'})
data.head()

,Master Record Number,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,Location,Date
0,902363382,2015,01,05,Weekday,0000,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356",2015-01-05
1,902364268,2015,01,06,Weekday,1500,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848",2015-01-06
2,902364412,2015,01,06,Weekend,2300,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006",2015-01-06
3,902364551,2015,01,07,Weekend,0900,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635",2015-01-07
4,902364615,2015,01,07,Weekend,1100,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482",2015-01-07


For Data Analysis take subset data only 2015 year: 

In [4]:
# Subset only 2015 year
subset_df = data[data['Year'] == 2015]

print(subset_df.shape)
#subset_df.head(20)

#save subset as raw data
subset_df.to_csv('subset2015.csv', index=False)

(3835, 14)


In [5]:
#import subset2015 
df = pd.read_csv("subset2015.csv", encoding="unicode_escape")

print(df.shape)
df.head()

(3835, 14)


,Master Record Number,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,Location,Date
0,902363382,2015,1,5,Weekday,0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356",2015-01-05
1,902364268,2015,1,6,Weekday,1500,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848",2015-01-06
2,902364412,2015,1,6,Weekend,2300,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006",2015-01-06
3,902364551,2015,1,7,Weekend,900,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635",2015-01-07
4,902364615,2015,1,7,Weekend,1100,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482",2015-01-07


In [6]:
# Remove Year, Month and Day columns
df = df.drop(columns=['Year', 'Month', 'Day'])

df = df[['Master Record Number','Date', 'Weekend?', 'Hour', 'Collision Type', 'Injury Type', 'Primary Factor', 'Reported_Location', 'Latitude', 'Longitude', 'Location']]

df.head()

,Master Record Number,Date,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,Location
0,902363382,2015-01-05,Weekday,0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356"
1,902364268,2015-01-06,Weekday,1500,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848"
2,902364412,2015-01-06,Weekend,2300,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006"
3,902364551,2015-01-07,Weekend,900,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635"
4,902364615,2015-01-07,Weekend,1100,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482"


Saving Data:

In [7]:
df.to_csv('data2015.csv', index=False)

Additional data is being loaded from the Yelp API, specifically gathering information about the number of bars in the location of car accidents.

In [8]:
#Function to load Additional Data from Yelp API 

def number_of_bars(row):
    
    api_key = 'OHBJD-6roYB0-z-lx_TwDukqiGPjGC2XtiE1SgeziEmMQDhWs26MfgUMRmsWB5DgHsR6M0NWfiLwlW9zihiXnDgCjsGW-Le67ybshU3_4YjIUThAMlm7mJ_4jOFBZHYx'
    headers = {'Authorization': f'Bearer {api_key}'}
    url_params = {
        'term': 'bars',        
        'limit': 50,
        'radius': 1500
    }

    url_params['latitude'] = row['Latitude']
    url_params['longitude'] = row['Longitude']

    res = requests.get(url='https://api.yelp.com/v3/businesses/search', headers=headers, params=url_params)
    data_bars = res.json()

    time.sleep(1)
    return data_bars['total']

In [9]:
#df['Number of Bars'] = df.apply(number_of_bars, axis=1)

#df.to_csv('data2015_Yelp-bars.csv', index=False)

#df

,Master Record Number,Date,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,Location,Number of Bars
0,902363382,2015-01-05,Weekday,0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356",41
1,902364268,2015-01-06,Weekday,1500,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848",42
2,902364412,2015-01-06,Weekend,2300,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006",2
3,902364551,2015-01-07,Weekend,900,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635",5
4,902364615,2015-01-07,Weekend,1100,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482",5
...,...,...,...,...,...,...,...,...,...,...,...,...
3830,902607777,2015-12-03,Weekday,1600,1-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,N DANIELS,39.169361,-86.596748,"39.16936111,-86.59674835",2
3831,902609951,2015-12-05,Weekday,1500,2-Car,No injury/unknown,OTHER (VEHICLE) - EXPLAIN IN NARRATIVE,SUMMIT & W 13TH,39.175152,-86.548048,"39.175152,-86.548048",30
3832,902616058,2015-12-04,Weekday,2100,2-Car,No injury/unknown,UNSAFE BACKING,RAUBS,39.229901,-86.635728,"39.22990074,-86.635728",1
3833,902623025,2015-12-04,Weekday,900,1-Car,No injury/unknown,ANIMAL/OBJECT IN ROADWAY,BURMA & SR37,39.313599,-86.511727,"39.31359897,-86.51172699",0


In [12]:
def add_weather_condition(df, api_key):
    
    df['Weather Condition'] = None
    weather_condition = []

    for index, row in df.iterrows():
        q = f"{row['Latitude']},{row['Longitude']}"
        date = f"{row['Date']}"
        res = requests.get(url=f"https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={api_key}&q={q}&format=json&date={date}")
        data_js = res.json()
        weather_data = data_js['data']['weather']
        
        for weather in weather_data:
            condition = weather['hourly'][0]['weatherDesc'][0]['value']
            weather_condition.append(condition)

    df.loc[:, 'Weather Condition'] = weather_condition
    
    print("done")

    time.sleep(1)

    return df

In [15]:
api_key = 'c9d927f6f8a04e07b9e222911231805'
df = add_weather_condition(df, api_key)

df.to_csv('data2015_API_bars_weather.csv', index=False)


KeyError: 'weather'